In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [39]:
#Quantidade maxima de páginas a serem extraidas
no_pages = 2

#Termo de busca digitado pelo user
search = "dan+brown"

def get_data(pageNo, searchTitle):  
    #Configuracao do BeatifulSoup
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    # MODELO DE URL DO SITE - https://busca.saraiva.com.br/busca?q=dan%20brown&page=2
    r = requests.get('https://busca.saraiva.com.br/busca?q='+str(searchTitle)+'&page='+ str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []

    #Encontra no html todas as lis com essa class e pega seu conteúdo
    for d in soup.findAll('li', attrs={'class':'nm-product-item'}):
        #print(d)
        name = d.find('a', attrs={'class':'nm-product-name'})
        #n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find('div', attrs={'class':'nm-product-sub'})
        try:
            author = author.text
            author = re.sub("\n| ","", author)
            author = re.split(',', author)
            if len(author) >= 2:
                author = author[1] + " " + author[0]
            else:
                author = author[0]
        except:
            continue
        #print(author)
        #rating = d.find('span', attrs={'class':'4,1 de 5 estrelas'})
        #r = rating.find_all('span', aria-label=True)
        
        price = d.find('div', attrs={'class':'nm-price-container'})
        try:
            price = price.text
            price = re.sub("\n| ", "", price)
            price = re.split('\$', price)
            price = "R$ " + price[1]
        except:
            break

        imgUrl = d.find('img', attrs={'class': 'nm-product-img'})['src']
        
        siteUrl = d.find('a', attrs={'class': 'nm-product-img-link'})['href']

        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(name.text)
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author)
            else:    
                all1.append('0')
                
        if imgUrl is not None:
            all1.append(imgUrl)
        else:
            all1.append("invalid-image")
        
        if siteUrl is not None:
            all1.append(siteUrl)
        else:
            all1.append("invalid-url-site")

        #if rating is not None:
            #print(rating.text)
        #    all1.append(rating.text)
        #else:
        #    all1.append('-1')

        #if users_rated is not None:
            #print(price.text)
        #    all1.append(users_rated.text)
        #else:
        #    all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [41]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i, search))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor', 'ImgUrl', 'SiteUrl', 'Preco'])
df.to_csv('saraiva_products.csv', index=False, encoding='utf-8')

In [42]:
df = pd.read_csv("saraiva_products.csv")
df.shape

(1, 5)

In [46]:
df.head(61)

,Titulo,Autor,ImgUrl,SiteUrl,Preco
0,Origem,Dan Brown,//lojasaraiva.vteximg.com.br/arquivos/ids/1211...,//www.saraiva.com.br/origem-9731880/p,"R$ 43,90"
